In [5]:
# Import necessary modules from PyTorch
import torchvision
import torch
from torchvision import transforms, datasets

## Preprosessing Data

In [7]:
# Data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define dataset root directory
data_dir = 'dataset_flower102/'

In [8]:
# Load train, validation, and test datasets
train_dataset = datasets.Flowers102(root=data_dir, split='train', transform=data_transforms, download=True)
valid_dataset = datasets.Flowers102(root=data_dir, split='val', transform=data_transforms, download=True)
test_dataset = datasets.Flowers102(root=data_dir, split='test', transform=data_transforms, download=True)

In [9]:
# Create data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## CNN Model Architecture

In [10]:
# Import necessary modules for Neural Network
import torch.nn as nn
import torch.optim as optim

In [12]:
# Define custom Convolution Neural Network
# Simple CNN architecture with two convolutional layers followed by max pooling, two fully connected layers, and a dropout layer for regularization.
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 56 * 56, 512)  # Adjust input size based on image dimensions
        self.fc2 = nn.Linear(512, num_classes)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)

    
    # Defines the forward pass of the network, where input data x is passed through each layer sequentially.
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        # Flatten the output from convolutional layers
        x = x.view(-1, 32 * 56 * 56)  # Adjust size based on image dimensions
        # Fully connected layers with dropout
        x = self.dropout(nn.ReLU()(self.fc1(x)))
        x = self.fc2(x)
        return x

In [13]:
# Create an instance of the CNN model
model = CNN(num_classes=102)  # 102 classes for Flowers102 dataset

## CNN Model Training

### Set Training Parameters

In [14]:
# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of training epochs
num_epochs = 10

### Model Training and Validation

In [16]:
for epoch in range(num_epochs):
    
    # Train the model
    model.train() # Set the model to training mode
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = model.forward(images) # Forward pass
        loss = loss_fn(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass
        optimizer.step() # Optimize

        train_loss += loss.item() * images.size(0) #  scalar value of the loss tensor for the current batch * the batch size to account for the loss per sample in the batch
        _, predicted = outputs.max(1) # Returns a tuple containing the maximum value along the specified dimension (class probabilities for each sample in the batch) and index of the max value
        total += labels.size(0) # Accumulates the total number of sample seen during training
        correct += predicted.eq(labels).sum().item() # Accumulates the total number of correct predictions over all batches.
    
    # Calculate training loss and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100.0 * correct / total

    # Print training loss and accuracy
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')



    # Validate the model
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model.forward(images)  # Forward pass
            loss = loss_fn(outputs, labels)  # Calculate the loss

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Calculate validation loss and accuracy
    val_loss = val_loss / len(valid_loader.dataset)
    val_accuracy = 100.0 * correct / total

    # Print validation loss and accuracy
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

Epoch 1/10, Training Loss: 6.1804, Training Accuracy: 1.76%
Epoch 1/10, Validation Loss: 4.5753, Validation Accuracy: 2.94%
Epoch 2/10, Training Loss: 4.3611, Training Accuracy: 4.41%
Epoch 2/10, Validation Loss: 4.1043, Validation Accuracy: 6.96%
Epoch 3/10, Training Loss: 3.7464, Training Accuracy: 10.78%
Epoch 3/10, Validation Loss: 3.7234, Validation Accuracy: 11.86%
Epoch 4/10, Training Loss: 3.0272, Training Accuracy: 25.20%
Epoch 4/10, Validation Loss: 3.5296, Validation Accuracy: 13.82%
Epoch 5/10, Training Loss: 2.0079, Training Accuracy: 51.57%
Epoch 5/10, Validation Loss: 3.4127, Validation Accuracy: 18.43%
Epoch 6/10, Training Loss: 0.9725, Training Accuracy: 76.86%
Epoch 6/10, Validation Loss: 3.4497, Validation Accuracy: 19.71%
Epoch 7/10, Training Loss: 0.4750, Training Accuracy: 88.14%
Epoch 7/10, Validation Loss: 3.5601, Validation Accuracy: 19.12%
Epoch 8/10, Training Loss: 0.2051, Training Accuracy: 95.20%
Epoch 8/10, Validation Loss: 3.6641, Validation Accuracy: 20.